<a href="https://colab.research.google.com/github/j-walker-red/shoes/blob/main/resize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 1
Instalar la librería para reconocimiento de caracteres, importar todas las librerías y cargar el modelo. Esto **solo hay que ejecutarlo una vez** cuando te conectas.

In [ ]:
# Instalamos la librería para reconocimiento de caracteres
!pip install easyocr

# Importamos librerías
import cv2
import easyocr
from google.colab import files
import numpy as np
import os
import zipfile

# Crear el lector de EasyOCR
reader = easyocr.Reader(['en', 'es'])  # Idiomas: inglés y español

# Paso 2
Solicitar el fichero ZIP con imágenes y procesarlas para redimensionar. Ejecútalo las veces que quieras en cada sesión.

In [ ]:
# Configuración
alto = 600
ancho = round(350/220*alto)  # La proporción es 350:220
margen_horizontal = 20  # Número de píxeles en blanco que dejamos a derecha e izquierda de la imagen
margen_vertical = 20  # Número de píxeles que dejamos en blanco arriba y abajo de la imagen

# Función para detectar si una imagen tiene texto
# Si tiene texto, devuelve la coordenada Y inferior del bounding box. Si no, devuelve 0
def detect_text(image_path):
    # Detectar texto y coordenadas
    results = reader.readtext(image_path)

    # Si no se detecta texto, devolver 0
    if not results:
        return 0

    # Cargar la imagen con OpenCV en escala de grises
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape  # Altura (h) y ancho (w) de la imagen

    # Encontrar la coordenada Y más cercana al borde inferior, solo para texto negro
    # Esto se hace para evitar que detecte texto que aparezca en las marcas de calzado
    lowest_y = 0
    for (coords, text, prob) in results:
        # Extraer las coordenadas del bounding box
        x_min = int(min(coord[0] for coord in coords))
        y_min = int(min(coord[1] for coord in coords))
        x_max = int(max(coord[0] for coord in coords))
        y_max = int(max(coord[1] for coord in coords))

        # Recortar el área del bounding box
        roi = gray[y_min:y_max, x_min:x_max]

        # Comprobar si el texto es negro sobre fondo blanco
        text_mean = np.mean(roi)
        if text_mean < 128:  # El texto es oscuro (promedio bajo)
            bg_mean = np.mean(gray[y_min-10:y_min, x_min:x_max])  # Supuesto fondo blanco (fuera del bounding box)
            if bg_mean > 200:  # El fondo es claro (promedio alto)
                max_y = max(coord[1] for coord in coords)  # Máximo valor de Y
                if max_y > lowest_y:
                    lowest_y = max_y

    return lowest_y

# Función que detecta el zapato en la imagen y cambia su tamaño
def resize_and_detect_shoe(image_path, output_path, target_width=ancho, target_height=alto):
    # Leer la imagen con OpenCV
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"No se pudo abrir la imagen: {image_path}")

    # Convertir a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar umbral adaptativo para separar el zapato del fondo
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Encontrar contornos en la imagen
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calculamos el bounding box que engloba a todos los contornos, por si hubiera más de un zapato
    if contours:
        x_min = float('inf')
        y_min = float('inf')
        x_max = float('-inf')
        y_max = float('-inf')

        # Recorrer todos los contornos para calcular el bounding box
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            x_max = max(x_max, x + w)
            y_max = max(y_max, y + h)

        # Coordenadas finales del bounding box
        x, y, w, h = x_min, y_min, x_max - x_min, y_max - y_min
    else:
        raise ValueError("No se detectó ningún contorno en la imagen")

    # Detectamos si existe texto. En ese caso ampliamos el bounding box inferior para que lo incluya
    lowest_y = detect_text(image_path)

    if lowest_y == 0:  # No existe texto
      shoe = image[y:y+h, x:x+w]  # Extraer el área del zapato
    else:
      shoe = image[y:lowest_y, x:x+w]

    # Redimensionar manteniendo proporciones
    shoe_h, shoe_w, _ = shoe.shape
    aspect_ratio = shoe_w / shoe_h
    target_width -= 2 * margen_horizontal
    target_height -= 2 * margen_vertical

    if aspect_ratio > (target_width / target_height):
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_height = target_height
        new_width = int(target_height * aspect_ratio)

    resized_shoe = cv2.resize(shoe, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

    # Crear un lienzo blanco del tamaño deseado con margen
    canvas = np.ones((target_height + 2 * margen_vertical, target_width + 2 * margen_horizontal, 3), dtype=np.uint8) * 255

    # Calcular los offsets para centrar
    x_offset = (target_width + 2 * margen_horizontal - new_width) // 2
    y_offset = (target_height + 2 * margen_vertical - new_height) // 2

    # Pegar el zapato redimensionado en el lienzo blanco
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_shoe

    # Guardar el resultado
    cv2.imwrite(output_path, canvas)

# Procesa las imágenes en el fichero comprimido
def process_images_in_zip(zip_path, output_zip_path, target_width=ancho, target_height=alto):
    # Crear un directorio temporal para extraer las imágenes
    temp_input_dir = '/content/temp_input_images'
    temp_output_dir = '/content/temp_output_images'
    os.makedirs(temp_input_dir, exist_ok=True)
    os.makedirs(temp_output_dir, exist_ok=True)

    # Extraer el archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_input_dir)

    # Procesar cada archivo en la carpeta extraída
    for filename in os.listdir(temp_input_dir):
        # Verificar si es un archivo de imagen válido
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            input_path = os.path.join(temp_input_dir, filename)
            output_filename = os.path.splitext(filename)[0] + ".jpg"
            output_path = os.path.join(temp_output_dir, output_filename)

            try:
                # Procesar la imagen
                resize_and_detect_shoe(input_path, output_path, target_width, target_height)
                print(f"Procesado: {filename} -> {output_filename}")
            except Exception as e:
                print(f"Error procesando {filename}: {e}")

    # Crear un archivo zip con las imágenes transformadas
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for root, _, files in os.walk(temp_output_dir):
            for file in files:
                zipf.write(os.path.join(root, file), file)

    # Limpiar los directorios temporales
    import shutil
    shutil.rmtree(temp_input_dir)
    shutil.rmtree(temp_output_dir)

# Procesar las imágenes
# Subir el archivo zip desde el disco duro
uploaded = files.upload()

# Procesar el archivo zip subido
for filename in uploaded.keys():
    process_images_in_zip(filename, '/content/output_images.zip')
    files.download('/content/output_images.zip')